# **Part 1: Understanding the Algorithm**


- **`Name`** : **Pavaris Asawakijtananont**

- **`Number`** : **65340500037**

In this homework, you have to implement 4 different function approximation-based RL algorithms:

- **Linear Q-Learning**

- **Deep Q-Network** (DQN)

- **REINFORCE algorithm**

- One algorithm chosen from the following Actor-Critic methods:
    - **Deep Deterministic Policy Gradient** (DDPG)
    - **Advantage Actor-Critic** (A2C)
    - **Proximal Policy Optimization** (PPO)
    - **Soft Actor-Critic** (SAC)

For each algorithm, describe whether it follows a `value-based, policy-based, or Actor-Critic approach`, specify the `type of policy it learns` (stochastic or deterministic), identify the type of `observation space and action space` (discrete or continuous), and `explain how each advanced RL method balances exploration and exploitation`.

ref 
- https://github.com/johnnycode8/gym_solutions
- https://arxiv.org/pdf/1312.5602
- https://huggingface.co/learn/deep-rl-course/unit3/deep-q-algorithm
- https://medium.com/@samina.amin/deep-q-learning-dqn-71c109586bae
- https://www.youtube.com/watch?v=EUrWGTCGzlA

## **Linear Q-Learning**

- Q-Learning we have to update throght weight instead of direct to Q value from learning to select the action
- and with linear function approximation we can use feature vector to directly update the action value or weight, this solution can handle with continuous value
 

#### **Updating Method**

We can use the weight value to approximate Q-value by use linear approximation equation, with equation

$$
Q(s,a) = \phi(s)^T w_a
$$

The weight can update by using gradient descent to converge to optimal action from state action pair, which we set the maximum action value along the state for all action to target policy add with reward value.

$$
w = w + \alpha[r + \gamma\max_{a'} Q_\pi(s',a') - Q(s,a)]X(s)
$$

- target policy (same with normal Q Learning) : $r + \gamma\max_{a'} Q_\pi(s',a')$
- direction of gradient : using the observation term(equal to gradient of action value along the weight) to define direction
    - Observation term $X(s) = \nabla_{w} Q(s,a ; w)$

#### **Question**
- **Approach Type** : Value Based
    - Directly estimated state-action value 
- **Policy Type** : Deterministic (with Stochastic Exploration)
    - Policy choosing the action that maximizes the estimate Q-Value from policy
    - In training process agent learn with epsilon-greedy(probability based)
- **Observation and Action Spaces**
    -   Observation Space: Linear Q-Learning is well-suited for environments with continuous observation spaces where `states are represented by feature vectors`.
    -   Action Space: Discrete only.
- **Balancing Exploration and Exploitation** : Epsilon-Greedy

ref 
- https://github.com/johnnycode8/gym_solutions
- https://arxiv.org/pdf/1312.5602
- https://huggingface.co/learn/deep-rl-course/unit3/deep-q-algorithm
- https://medium.com/@samina.amin/deep-q-learning-dqn-71c109586bae
- https://www.youtube.com/watch?v=EUrWGTCGzlA

## **Deep Q Network**

### **Updating Method**

- Deep Q network is use Neural Network as function approximator to approximate action value give the state, and same as the other Q Learning Algorithm we must have the target value to find loss function in this DQN we give target as:

$$
y = r_t + \gamma \max_a Q(s_{t+1} , a ; \theta)
$$


<div style="text-align: center;">
    <img src="images\Deep-Q-Learning-code.png" alt="Description" width="600">
</div>

- And updating the Policy network by calculating with MSE loss

$$
L(\theta)= (y_j - Q(\phi_{j} , a_j ; \theta))^2
$$

and updating weight with

$$
\theta \leftarrow \theta + \alpha \nabla (y_j +\gamma \max_{a'}Q(\phi_{j+1} , a' ; \theta))^2
$$
$$
\theta \leftarrow \theta + \alpha \nabla L(\theta)
$$

- DQN is difference from traditional Q Learning, DQN is collecting buffer from environment and update with batch size by updating with fixed time step

<div style="text-align: center;">
    <img src="images\DQN_overview.png" alt="Description" width="800">
</div>

<div style="text-align: center;">
    <img src="images\DQN_flow.png" alt="Description" width="600">
</div>

- to get action value DQN learning throught the weight in the neural network to find optimal weight
- to get the action from policy DQN using deterministic policy (argument max) to find optimal value from action value
- and get experience from environment, store into replay memory to updating both network(policy and target)

#### **Target Network Updating**

- updating from experience is sampling efficient method and can updating multiple time same experience or transition , to if only use one policy if we it like to step the target futher cause of correlation of the value (from same network)

$$
 \mathbb{E}_{(s,a,r,s')~U(D)}[((r + \gamma \max_a Q(s',a;\theta) - Q(s,a;\theta))^2)]
$$

- to solve this problem target network come out to, instead we using the same network we using the seperate network to act as target
- The target network is must updating same as policy network
    - to updating target we can set $\theta^- = \theta$ (updating weight of target network equal policy network) with fix timestep
    - another method is soft updating with updating target network every timestep following the equation, when tau is small value to converge target network to policy
        $$
        \theta^{-} = (\tau)\theta + (1-\tau)\theta^{-}
        $$



#### **Question**
- **Approach Type** : Value Based
- **Policy Type** : Deterministic
    - Policy choosing the action that maximizes the estimate Q-Value from policy
- **Observation and Action Spaces**
    -   Observation Space: discrete or continuous
    -   Action Space: Discrete only
- **Balancing Exploration and Exploitation**
    - Epsilon-greedy : decaying exploration rate over time
    - Experience Replay : store experience buffer and sample mini-batches to update, this method break temporal correlation
    

ref
- https://thammasorn.github.io/2020/06/03/DQN.html
- https://skrl.readthedocs.io/en/latest/api/agents/dqn.html
- https://medium.com/@samina.amin/deep-q-learning-dqn-71c109586bae
- https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html ***

## **REINFORCE**

- REINFORCE is policy based reinforcement learning and it is the policy gradient algorithm, base on policy
- policy gradient is try to updating neural network parameter to estimate probability of action given the state to get maximize reward  


#### **Stochastic Policy**
- In policy gradient we using the stochastic policy to optimize the objective function
    - this is different from deterministic policy that we will have probabilibty to get every action, this open the chance to more exploration with probability to select other action

<div style="text-align: center;">
    <img src="images\deterministic-vs-stochastic.png" alt="Description" width="750">
</div>

- to implement stochastic policy in neural network we can applying `Softmax Function` to output with probability density function

### **Gradient Ascent**
- to calculate objective function of REINFORCE is using the reward directly to be a gradient but reward is scalar value that couldn't find the gradient
$$
\mathbb{E} [\sum_{t+1}^{\infty}r_t] = \frac{\sum_{\tau}\sum_{t+1}^{\infty}r_t^\tau}{N}
$$

this equation mean expected return is mean of from reward from all transition state($\tau$) everytime step

$$
\mathbb{E} [\sum_{t+1}^{\infty}r_t] = \sum_\tau P(\tau;\theta)R(\tau)
$$

and we must use some trick(log likelihood trick) to transfrom in to derivativable form

$$
\nabla \sum_\tau P(\tau;\theta)R(\tau) =  \sum_\tau \nabla P(\tau;\theta)R(\tau) = \sum_\tau P(\tau;\theta) \frac{\nabla P(\tau;\theta)}{P(\tau;\theta)} R(\tau)
$$

and we will get the final form of objective function to updating the network

$$
\sum_\tau P(\tau;\theta) \nabla_{\theta} \log P(\tau;\theta) R(\tau) = \mathbb{E}_{\tau \sim \pi_{\tau}} \nabla_{\theta} \log P(\tau;\theta) R(\tau)
$$

### **Updating Method**



- REINFORCE algorithm is same as Monte Carlo method to sampling the trajectory to collect experience and then rerun the trajectory to calculate return each time step and multiply with log probability following the equation and will get loss to backpropagate the network

<div style="text-align: center;">
    <img src="images\reinforce-code.png" alt="Description" width="750">
</div>

#### **Question**
- **Approach Type** : Policy-based 
- **Policy Type** : Stochastic Policy
    - the policy outputs a probability distribution over actions
- **Observation and Action Spaces**
    -   Observation Space: discrete or continuous
    -   Action Space: discrete or continuous
- **Balancing Exploration and Exploitation**
    - Stochastic Policy : normally random the action from probability distribution over action
    

ref 
- https://thammasorn.github.io/2020/07/30/reinforce.html

## **Actor Critic : Proximal Policy Optimization**

- PPO is policy gradient method which alternate sanpling data from environment that interacton with and optimizing `surrogate` objective function with using stochastic gradient ascent
- PPO have some benefit from TRPO but more general and simpler to implement 

### **Policy Optimization**
##### **Policy Gradient**
- loss of policy gradient is given
$$
L^{PG} = \hat{\mathbb{E}}_t [\log \pi_\theta (a_t | s_t) \hat{A_t}]
$$

- this loss is same as the REINFORCE

##### **Clipping Surrogate Objective**
- TRPO is maximize a `surrogate` objective given

$$
r_t(\theta) = \frac{\pi_{\theta}(a_t | s_t)}{\pi_{\theta_{old}}(a_t | s_t)}
$$
$$
L^{CLIP} = \hat{\mathbb{E}}_t [ \min ( r_t(\theta) \hat{A_t}  , clip(r_t(\theta) , 1-\epsilon , 1+\epsilon )\hat{A_t} )]
$$

<div style="text-align: center;">
    <img src="images\PPO-clipping.png" alt="Description" width="750">
</div>

- where epsilon is a hyperparameter, modifies the surrogateobjective by clipping the probability ratio, which removes the incentive for moving $r_t$ outside of the interval [1 − $\epsilon$, 1 + $\epsilon$]

##### **Final Loss**
- contain with 3 term of loss

$$
L_t^{CLIP+VF+S}(\theta) = \hat{\mathbb{E}}_t [ L_t^{CLIP}(\theta) - c_1 L_t^{VF}(\theta) + c_2 S[\pi_{\theta}](s_t) ]
$$

- c1 , c2 is constant value to multiplying with loss term
- last term is `Entropy` bonus if calculate from entropy of output action is term can increase more exploration on agent

### **Updating Method**

#### **Advantage Calculation**


- In loss term we need to find `Advantages` to define how that action is good, **`GAE` [generalized advantage estimation]** is the choice in PPO that reduces the advantages full equation form

$$
\hat{A}_t = -V(s_t)+r_t + \gamma r_{t+1} + ... + \gamma^{T-t+1}r_{T-1} + \gamma^{T-t}V(s_T)
$$

to this form

$$
\hat{A}_t = \zeta_{t} + (\gamma \lambda)\zeta_{t+1} +...+...+(\gamma \lambda)^{T-t+1} \zeta_{T-1}
$$
where 
$$
\zeta_t = r_t +\gamma(s_{t+1})-V(s_t)
$$

#### **Collect Trajectory and Optimize surrogate**

<div style="text-align: center;">
    <img src="images\PPO-code.png" alt="Description" width="750">
</div>

- PPO is collecting sampling from **`Parallel actors`** and collect transition with fixed timestep, and then optimize surrogate loss on the `NT` timesteps ( Number of actor * timestep) with `K` epoch and optimize with batch size `M` when M $\le$ NT

> More Detail more in PART 2 

#### **Question**
- **Approach Type** : Policy-based / Actor - Critic
- **Policy Type** : Stochastic Policy
    - the policy outputs a probability distribution over actions
- **Observation and Action Spaces**
    -   Observation Space: discrete or continuous
    -   Action Space: discrete or continuous
- **Balancing Exploration and Exploitation**
    - Stochastic Policy : normally random the action from probability distribution over action
    - Clipping : Prevent excessive changes that might swing the policy too rapidly toward exploitation
    - Entropy Bonus : This bonus encourages continuous exploration even as the policy improves.
    

ref
- https://arxiv.org/pdf/1707.06347
- https://joel-baptista.github.io/phd-weekly-report/posts/ac/
- https://github.com/saqib1707/RL-PPO-PyTorch/blob/main/src/ppo.py
- https://github.com/nikhilbarhate99/PPO-PyTorch/blob/master/PPO.py
- https://www.youtube.com/watch?v=MEt6rrxH8W4
- https://medium.com/@eyyu/coding-ppo-from-scratch-with-pytorch-part-2-4-f9d8b8aa938a ***

Exploration vs. Exploitation

PPO trains a stochastic policy in an on-policy way. This means that it explores by sampling actions according to the latest version of its stochastic policy. The amount of randomness in action selection depends on both initial conditions and the training procedure. Over the course of training, the policy typically becomes progressively less random, as the update rule encourages it to exploit rewards that it has already found. This may cause the policy to get trapped in local optima